In [1]:
import pickle
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [4]:
s_data = pickle.load(open("sensitivity.p", "rb"))

In [6]:
s_data.keys()

dict_keys(['p0', 'b0', 'c0', 's0', 'dels+', 'delb+', 'dels-', 'delc-', 'delb-', 'pi0', 'ap', 'lambda', 'u0', 'd0', 'ec0', 'es0', 'r0'])